In [11]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA

In [2]:
spark=SparkSession.builder.appName("pca").getOrCreate()
data=spark.read.csv("D:\\21bad014_ML\\Rice_Sample.csv",inferSchema=True,header=True)

In [3]:
data.show()

+-----+---------+----------+----------+------------+--------+--------+-----------+------+------------+---------+-----------+-------------+-------------+-------------+-------------+--------+--------+--------+--------+--------+--------+-------+-------+-------+----------+----------+----------+-----------+-----------+-----------+------+------+------+-------+-------+-------+--------+-------+-------+---------+---------+---------+---------+--------+---------+--------+--------+--------+-------+-------+-------+-------+-------+-------+---------+---------+---------+-----------+-----------+-----------+--------+--------+--------+-------+--------+--------+-------+-------+-------+---------+----------+----------+-----------+-----------+-----------+------+------+------+--------+--------+--------+-------+-------+-------+----------+----------+----------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+----------+---------

In [4]:
#LABEL ENCODING
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="CLASS", outputCol="Class_label") 
new_df = indexer.fit(data).transform(data)
new_df=new_df.drop("CLASS").withColumnRenamed("Class_label","Class")
new_df.show()

+-----+---------+----------+----------+------------+--------+--------+-----------+------+------------+---------+-----------+-------------+-------------+-------------+-------------+--------+--------+--------+--------+--------+--------+-------+-------+-------+----------+----------+----------+-----------+-----------+-----------+------+------+------+-------+-------+-------+--------+-------+-------+---------+---------+---------+---------+--------+---------+--------+--------+--------+-------+-------+-------+-------+-------+-------+---------+---------+---------+-----------+-----------+-----------+--------+--------+--------+-------+--------+--------+-------+-------+-------+---------+----------+----------+-----------+-----------+-----------+------+------+------+--------+--------+--------+-------+-------+-------+----------+----------+----------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+----------+---------

In [5]:
from pyspark.sql.types import IntegerType
new_df = new_df.withColumn("Class", new_df["Class"].cast(IntegerType()))

In [6]:
new_df.show()

+-----+---------+----------+----------+------------+--------+--------+-----------+------+------------+---------+-----------+-------------+-------------+-------------+-------------+--------+--------+--------+--------+--------+--------+-------+-------+-------+----------+----------+----------+-----------+-----------+-----------+------+------+------+-------+-------+-------+--------+-------+-------+---------+---------+---------+---------+--------+---------+--------+--------+--------+-------+-------+-------+-------+-------+-------+---------+---------+---------+-----------+-----------+-----------+--------+--------+--------+-------+--------+--------+-------+-------+-------+---------+----------+----------+-----------+-----------+-----------+------+------+------+--------+--------+--------+-------+-------+-------+----------+----------+----------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+----------+---------

In [7]:
features=new_df.columns[:-1]

In [8]:
assembler=VectorAssembler(inputCols=features,outputCol='features')
new_df=assembler.transform(new_df)
new_df.show()

+-----+---------+----------+----------+------------+--------+--------+-----------+------+------------+---------+-----------+-------------+-------------+-------------+-------------+--------+--------+--------+--------+--------+--------+-------+-------+-------+----------+----------+----------+-----------+-----------+-----------+------+------+------+-------+-------+-------+--------+-------+-------+---------+---------+---------+---------+--------+---------+--------+--------+--------+-------+-------+-------+-------+-------+-------+---------+---------+---------+-----------+-----------+-----------+--------+--------+--------+-------+--------+--------+-------+-------+-------+---------+----------+----------+-----------+-----------+-----------+------+------+------+--------+--------+--------+-------+-------+-------+----------+----------+----------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+----------+---------

In [9]:
from pyspark.ml.feature import StandardScaler
sc = StandardScaler(inputCol="features",
                        outputCol="scaled_val",
                        withStd=True,
                        withMean=False)
scalerModel=sc.fit(new_df)
new_df= scalerModel.transform(new_df)

In [13]:
selected_df=new_df.select("features","scaled_val").show()

+--------------------+--------------------+
|            features|          scaled_val|
+--------------------+--------------------+
|[7805.0,437.915,2...|[2.56728926422790...|
|[7503.0,340.757,1...|[2.46795276739295...|
|[5124.0,314.617,1...|[1.68543115821957...|
|[7990.0,437.085,2...|[2.62814109175925...|
|[7433.0,342.893,1...|[2.44492775157027...|
|[11648.0,445.527,...|[3.83136263289258...|
|[7621.0,450.325,2...|[2.50676636549402...|
|[8582.0,367.338,1...|[2.82286693985956...|
|[5450.0,320.362,1...|[1.79266194619373...|
|[6781.0,307.023,1...|[2.23046617562196...|
|[9656.0,451.287,2...|[3.17613646833884...|
|[8460.0,362.862,1...|[2.78273762656862...|
|[7090.0,401.778,1...|[2.33210517403918...|
|[6208.0,300.124,1...|[2.04198997467352...|
|[7116.0,411.103,1...|[2.34065732277332...|
|[6276.0,288.886,1...|[2.06435713290126...|
|[5832.0,282.75,10...|[1.91831274682602...|
|[6628.0,311.431,1...|[2.18014006960955...|
|[5251.0,327.214,1...|[1.72720511549785...|
|[12769.0,447.83,1...|[4.2000918

In [16]:
pca = PCA(k=3,inputCol="scaled_val",outputCol="PCA_FEATURES")
model = pca.fit(new_df)
new_df = model.transform(new_df)
new_df.show()

+-----+---------+----------+----------+------------+--------+--------+-----------+------+------------+---------+-----------+-------------+-------------+-------------+-------------+--------+--------+--------+--------+--------+--------+-------+-------+-------+----------+----------+----------+-----------+-----------+-----------+------+------+------+-------+-------+-------+--------+-------+-------+---------+---------+---------+---------+--------+---------+--------+--------+--------+-------+-------+-------+-------+-------+-------+---------+---------+---------+-----------+-----------+-----------+--------+--------+--------+-------+--------+--------+-------+-------+-------+---------+----------+----------+-----------+-----------+-----------+------+------+------+--------+--------+--------+-------+-------+-------+----------+----------+----------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+----------+---------

In [17]:
pca_results=new_df.select("features","scaled_val","PCA_FEATURES").show()

+--------------------+--------------------+--------------------+
|            features|          scaled_val|        PCA_FEATURES|
+--------------------+--------------------+--------------------+
|[7805.0,437.915,2...|[2.56728926422790...|[-25.910075939834...|
|[7503.0,340.757,1...|[2.46795276739295...|[-20.196368872331...|
|[5124.0,314.617,1...|[1.68543115821957...|[-23.743976200814...|
|[7990.0,437.085,2...|[2.62814109175925...|[-26.864270351167...|
|[7433.0,342.893,1...|[2.44492775157027...|[-20.768427530101...|
|[11648.0,445.527,...|[3.83136263289258...|[-36.036456348054...|
|[7621.0,450.325,2...|[2.50676636549402...|[-25.705351703702...|
|[8582.0,367.338,1...|[2.82286693985956...|[-23.099274911128...|
|[5450.0,320.362,1...|[1.79266194619373...|[-24.674904731855...|
|[6781.0,307.023,1...|[2.23046617562196...|[-20.526781121252...|
|[9656.0,451.287,2...|[3.17613646833884...|[-26.957101724128...|
|[8460.0,362.862,1...|[2.78273762656862...|[-25.912534459114...|
|[7090.0,401.778,1...|[2.

In [18]:
print('Explained Variance Ratio', model.explainedVariance.toArray())

Explained Variance Ratio [0.40617946 0.16382906 0.10412435]


In [ ]:
spark.stop()